IMPORT LIBRARIES

In [42]:
import re
import nltk
import math
from nltk.tokenize import word_tokenize

Cleaning the Text

In [43]:
def clean_text(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []
    # removing special characters and extra whitespaces
    for sentence in article:
        sentence = re.sub('[^a-zA-Z]',' ',str(sentence))
        sentence = re.sub('[\s+]',' ', sentence)
        sentences.append(sentence)
    sentences.pop()
    display = " ".join(sentences)
    print('Initial Text: ')
    print(display)
    print('\n')
    return sentences

Number of Words in Each Sentence

In [44]:
def count_words(sent):
    count = 0
    words = word_tokenize(sent)
    for word in words:
        count = count + 1
    return count

In [45]:
# getting data about each sentence (frequency of words)
def count_in_sentence(sentences):
    txt_data = []
    for index,sentence in enumerate(sentences):
        count = count_words(sentence)
        temp = {'id' : index, 'word_cnt' : count}
        txt_data.append(temp)
    return txt_data

Creating List of Frequencies for each Word in all Documents


In [46]:
def freq_dict(sentences):
    freq_list = []
    for index,sentence in enumerate(sentences):
        freq_dict = {}
        words = word_tokenize(sentence)
        for word in words:
            word = word.lower()
            freq_dict[word] = freq_dict.get(word,0) + 1
        temp = {'id' : index, 'freq_dict' : freq_dict}
        freq_list.append(temp)
    return freq_list

Calculating TF and IDF Values

In [47]:
def calc_TF(text_data, freq_list):
    tf_scores = []
    for item in freq_list:
        ID = item['id']
        for key,value in item['freq_dict'].items():
            temp = {
                'id': item['id'],
                'tf_score': value/text_data[ID]['word_cnt'],
                'key': k
            }
            tf_scores.append(temp)
    return tf_scores

In [48]:
def calc_IDF(text_data, freq_list):
    idf_scores = []
    for index,item in enumerate(freq_list):
        for key in item['freq_dict']:
            val = sum([k in it['freq_dict'] for it in freq_list])
            temp = {
                'id':index,
                'idf_score': math.log(len(text_data)/(val+1)),
                'key': key
            }
            idf_scores.append(temp)
    return idf_scores

Calculating TF-IDF Values

In [49]:
def calc_TFIDF(tf_scores,idf_scores):
    tfidf_scores = []
    for idf in idf_scores:
        for tf in tf_scores:
            if(idf['key'] == tf['key'] and idf['id'] == tf['id']):
                temp = {
                    'id': idf['id'],
                    'tfidf_score': idf['idf_score']*tf['tf_score'],
                    'key': idf['key']
                }
                tfidf_scores.append(temp)
    return tfidf_scores

Ranking all the Sentences

In [50]:
def sent_scores(tfidf_scores, sentences, text_data):
    sent_data = []
    for txt in text_data:
        score = 0 
        for index in range(len(tfidf_scores)):
            t_dict = tfidf_scores[index]
            if(txt['id'] == t_dict['id']):
                score = score + t_dict['tfidf_score']
        temp = {
            'id': txt['id'],
            'score': score,
            'sentence': sentences[txt['id']]
        }
        sent_data.append(temp)
    return sent_data

In [51]:
def summary(sent_data):
    count = 0
    summary = []
    for t_dict in sent_data:
        count = count + t_dict['score']
    avg = count / len(sent_data)
    for sent in sent_data:
        if(sent['score'] >= avg*0.9):
            summary.append(sent['sentence'])
    summary = ". ".join(summary)
    return summary

Runtime

In [57]:
sentences = clean_text('input.txt')
text_data = count_in_sentence(sentences)

freq_list = freq_dict(sentences)
tf_scores = calc_TF(text_data, freq_list)
idf_scores = calc_IDF(text_data, freq_list)

tfidf_scores = calc_TFIDF(tf_scores, idf_scores)

sent_data = sent_scores(tfidf_scores, sentences, text_data)
result = summary(sent_data)

print('SUMMARY:')
print(result)

Initial Text: 
Napoleon Hill is the most famous conman you ve probably never heard of Born into poverty in rural Virginia at the end of the   th century  Hill went on to write one of the most successful self help books of the   th century  Think and Grow Rich In fact  he helped invent the genre But it s the untold story of Hill s fraudulent business practices  tawdry sex life  and membership in a New York cult that makes him so fascinating That cult would become infamous in the late     s for trying to raise an  immortal baby   But even those who know the story of Immortal Baby Jean may not know that the cult was inspired by Hill s teachings  practically using his most famous work as their holy text Don t worry  the whole story of Napoleon Hill only gets weirder from there Modern readers are probably familiar with the      sensation The Secret  but the concepts in that book were essentially plagiarized from Napoleon Hill s      classic Think and Grow Rich  which has reportedly sold ove